# 8.2 일대일 병합
* 수평결합 : 데이터 테이블의 열을 다른 테이블의 열과 병합하는 것
    - SQL join 연산에 해당
    - 1. 일대일 병합 : 양쪽 파일 모두에서 병합 기준 열에 중복값이 없음
    - 2. 일대다 병합 : 오른쪽 데이터 테이블의 병합 기준 열에 중복이 있음
    - 3. 다대다 병합 : 왼쪽, 오른쪽 데이터 테이블 모두에서 병합 기준 열에 중복 있음
* 용어는 '병합 기준 열(병합 기준 값, merge-by value)'으로 통일하여 사용
    - 관계형 데이터베이스(RDB)의 기본 키, 외래 키 개념 배제
        - 관계형 시스템에서 데이터 추출 시 위의 개념을 파악하면 도움되고, 판다스 인덱스를 설정할 때도 고려해야 하는 부분임
        - 그러나 데이터 전처리 과정 중 병합을 하다보면 이러한 키의 범위를 넘어설때가 많음
        
* 유용한 코드📌
    - **join** 
        - `{left dataframe}.join({right dataframe})` : 왼쪽/오른쪽 데이터프레임 양쪽 모두의 index가 병합 기준으로 사용될 수 있는 상태에서 가장 효율적인 코드
        - `{left dataframe}.join({right dataframe}.set_index(['병합 기준 열']))` : 왼쪽 데이터프레임의 index를 병합 기준으로 사용 시 
        - `{left dataframe}.join({right dataframe}, on=['병합 기준 열']))` : 왼쪽/오른쪽 모두 index가 기준으로 사용될 수 없을 때
    - **merge**
        - `pd.merge({left dataframe}, {right dataframe}, on=['병합 기준 열'], how='left')` : 병합 기준 열, 병합 방식을 직접 설정하는게 빠를 때
    - join 함수는 merge() 함수를 기반으로 만들어졌으며 작동방식이 비슷하지만 행 인덱스를 기준으로 데이터프레임을 병합한다는 점에서 차이가 있다.(출처 : https://kimdingko-world.tistory.com/207)  
      즉, join은 on 매개변수 사용하지 않고도 병합이 가능함

## 준비
* NLS 데이터셋 2개 사용
    - 한 개의 행(row) = 한 사람의 데이터
    - 1st file : 취업, 학력, 소득
    - 2nd file : 응답자의 부모의 소득, 학력

## 8.2.1 left join

### 1. 데이터 로드

In [118]:
import pandas as pd
import os
os.listdir('data')

['cmacitations.csv',
 'cmacitations.pkl',
 'cmacreators.csv',
 'cmacreators.pkl',
 'ghcn',
 'ltcountries.csv',
 'ltcountries.pkl',
 'ltcountry',
 'ltlocations.csv',
 'ltlocations.pkl',
 'nls97add.csv',
 'nls97add.pkl',
 'nls97colenr.csv',
 'nls97colenr.pkl',
 'nls97f.csv',
 'nls97f.pkl',
 'nls97weeksworked.csv',
 'nls97weeksworked.pkl']

In [119]:
# 1st file
nls97 = pd.read_csv('data/nls97f.csv')
nls97.set_index('personid', inplace=True)
# 2nd file
nls97add = pd.read_csv('data/nls97add.csv')

In [120]:
nls97.head()

,gender,birthmonth,birthyear,highestgradecompleted,maritalstatus,childathome,childnotathome,wageincome,weeklyhrscomputer,weeklyhrstv,...,colenroct13,colenrfeb14,colenroct14,colenrfeb15,colenroct15,colenrfeb16,colenroct16,colenrfeb17,colenroct17,originalid
personid,,,,,,,,,,,,,,,,,,,,,
100061,Female,5,1980,13.0,Married,4.0,0.0,12500.0,10 hours or more a week,11 to 20 hours a week,...,1. Not enrolled,1. Not enrolled,1. Not enrolled,1. Not enrolled,1. Not enrolled,1. Not enrolled,1. Not enrolled,1. Not enrolled,1. Not enrolled,8245
100139,Male,9,1983,12.0,Married,2.0,0.0,120000.0,1 to 3 hours a week,3 to 10 hours a week,...,1. Not enrolled,1. Not enrolled,1. Not enrolled,1. Not enrolled,1. Not enrolled,1. Not enrolled,1. Not enrolled,1. Not enrolled,1. Not enrolled,3962
100284,Male,11,1984,7.0,Never-married,1.0,0.0,58000.0,None,11 to 20 hours a week,...,1. Not enrolled,1. Not enrolled,1. Not enrolled,1. Not enrolled,1. Not enrolled,1. Not enrolled,1. Not enrolled,1. Not enrolled,1. Not enrolled,3571
100292,Male,4,1982,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1. Not enrolled,1. Not enrolled,1. Not enrolled,1. Not enrolled,1. Not enrolled,NaN,NaN,NaN,NaN,2979
100583,Male,1,1980,13.0,Married,4.0,0.0,30000.0,Less than 1 hour a week,3 to 10 hours a week,...,1. Not enrolled,1. Not enrolled,1. Not enrolled,1. Not enrolled,1. Not enrolled,1. Not enrolled,1. Not enrolled,1. Not enrolled,1. Not enrolled,8511


In [121]:
nls97.columns

Index(['gender', 'birthmonth', 'birthyear', 'highestgradecompleted',
       'maritalstatus', 'childathome', 'childnotathome', 'wageincome',
       'weeklyhrscomputer', 'weeklyhrstv', 'nightlyhrssleep', 'satverbal',
       'satmath', 'gpaoverall', 'gpaenglish', 'gpamath', 'gpascience',
       'highestdegree', 'govprovidejobs', 'govpricecontrols', 'govhealthcare',
       'govelderliving', 'govindhelp', 'govunemp', 'govincomediff',
       'govcollegefinance', 'govdecenthousing', 'govprotectenvironment',
       'weeksworked00', 'weeksworked01', 'weeksworked02', 'weeksworked03',
       'weeksworked04', 'weeksworked05', 'weeksworked06', 'weeksworked07',
       'weeksworked08', 'weeksworked09', 'weeksworked10', 'weeksworked11',
       'weeksworked12', 'weeksworked13', 'weeksworked14', 'weeksworked15',
       'weeksworked16', 'weeksworked17', 'colenrfeb97', 'colenroct97',
       'colenrfeb98', 'colenroct98', 'colenrfeb99', 'colenroct99',
       'colenrfeb00', 'colenroct00', 'colenrfeb01', 'col

In [122]:
nls97.shape

(8984, 89)

In [123]:
nls97add.head()

,originalid,motherage,parentincome,fatherhighgrade,motherhighgrade
0,1,26,-3,16,8
1,2,19,-4,17,15
2,3,26,63000,-3,12
3,4,33,11700,12,12
4,5,34,-3,12,12


In [124]:
nls97add.shape

(8984, 5)

### 2. 유일값 개수 - 행 수 같은지 확인

In [125]:
nls97.originalid.nunique()==nls97.shape[0]

True

In [126]:
nls97add.originalid.nunique()==nls97add.shape[0]

True

### 3. 불일치하는 ID 생성
- 실습 목적에 따라 고의로 훼손함
- originalid는 nls97 파일 마지막열, nls97add 파일 첫 번째 열임

In [127]:
nls97.sort_values('originalid')

,gender,birthmonth,birthyear,highestgradecompleted,maritalstatus,childathome,childnotathome,wageincome,weeklyhrscomputer,weeklyhrstv,...,colenroct13,colenrfeb14,colenroct14,colenrfeb15,colenroct15,colenrfeb16,colenroct16,colenrfeb17,colenroct17,originalid
personid,,,,,,,,,,,,,,,,,,,,,
135335,Female,9,1981,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1. Not enrolled,1. Not enrolled,1. Not enrolled,1. Not enrolled,1. Not enrolled,NaN,NaN,NaN,NaN,1
999406,Male,7,1982,14.0,Never-married,NaN,NaN,116000.0,4 to 6 hours a week,3 to 10 hours a week,...,1. Not enrolled,1. Not enrolled,1. Not enrolled,1. Not enrolled,1. Not enrolled,1. Not enrolled,1. Not enrolled,1. Not enrolled,1. Not enrolled,2
151672,Female,9,1983,16.0,Married,2.0,0.0,NaN,10 hours or more a week,3 to 10 hours a week,...,3. 4-year college,3. 4-year college,3. 4-year college,3. 4-year college,1. Not enrolled,1. Not enrolled,1. Not enrolled,1. Not enrolled,1. Not enrolled,3
750699,Female,2,1981,13.0,Never-married,1.0,1.0,45000.0,1 to 3 hours a week,3 to 10 hours a week,...,1. Not enrolled,1. Not enrolled,1. Not enrolled,1. Not enrolled,1. Not enrolled,1. Not enrolled,1. Not enrolled,1. Not enrolled,1. Not enrolled,4
781297,Male,10,1982,12.0,Married,2.0,0.0,125000.0,Less than 1 hour a week,3 to 10 hours a week,...,1. Not enrolled,1. Not enrolled,1. Not enrolled,1. Not enrolled,1. Not enrolled,1. Not enrolled,1. Not enrolled,1. Not enrolled,1. Not enrolled,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
505861,Female,3,1980,10.0,Married,0.0,3.0,17500.0,4 to 6 hours a week,3 to 10 hours a week,...,1. Not enrolled,1. Not enrolled,1. Not enrolled,1. Not enrolled,1. Not enrolled,1. Not enrolled,1. Not enrolled,1. Not enrolled,1. Not enrolled,9018
368078,Male,9,1984,14.0,Never-married,0.0,4.0,35000.0,Less than 1 hour a week,3 to 10 hours a week,...,1. Not enrolled,1. Not enrolled,1. Not enrolled,1. Not enrolled,1. Not enrolled,1. Not enrolled,1. Not enrolled,1. Not enrolled,1. Not enrolled,9019
215605,Male,7,1980,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3. 4-year college,1. Not enrolled,1. Not enrolled,1. Not enrolled,1. Not enrolled,NaN,NaN,NaN,NaN,9020


In [128]:
nls97 = nls97.sort_values('originalid')
nls97add = nls97add.sort_values('originalid')
nls97.iloc[0:2, -1] = nls97[0:2].originalid+10000
nls97.originalid.head(3)

personid
135335    10001
999406    10002
151672        3
Name: originalid, dtype: int64

In [129]:
nls97add.iloc[0:2, 0] = nls97add[0:2].originalid+20000
nls97add.originalid.head(3)

0    20001
1    20002
2        3
Name: originalid, dtype: int64

### 4. join으로 왼쪽 조인
* join()은 행 인덱스를 기준으로 결합하는 함수이다.
* 일치하는 originalid가 없는 행들은 오른쪽 데이터프레임 열의 값들이 모두 누락된다.
    * nls97 = 왼쪽 데이터프레임
    * nls97add = 오른쪽 데이터프레임


In [130]:
nlsnew.index

Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
            ...
            8972, 8973, 8974, 8975, 8976, 8977, 8978, 8979, 8980, 8981],
           dtype='int64', length=8982)

In [131]:
# 책에 제시된 코드 실행 시
nlsnew = nls97.join(nls97add.set_index(['originalid']))
nlsnew.loc[:,['originalid', 'gender', 'birthyear', 'motherage', 'parentincome']]
# nlsnew.loc[nlsnew.originalid>9999, ['originalid','gender','birthyear','motherage','parentincome']]

,originalid,gender,birthyear,motherage,parentincome
personid,,,,,
135335,10001,Female,1981,NaN,NaN
999406,10002,Male,1982,NaN,NaN
151672,3,Female,1983,NaN,NaN
750699,4,Female,1981,NaN,NaN
781297,5,Male,1982,NaN,NaN
...,...,...,...,...,...
505861,9018,Female,1980,NaN,NaN
368078,9019,Male,1984,NaN,NaN
215605,9020,Male,1980,NaN,NaN


In [132]:
# 올바르다고 생각되는 코드
nlsnew2 = nls97.set_index(['originalid']).join(nls97add.set_index(['originalid']))
nlsnew2.loc[:,['gender', 'birthyear', 'motherage', 'parentincome']]
# nlsnew2.loc[nlsnew2.index>9999,['gender', 'birthyear', 'motherage', 'parentincome']]

,gender,birthyear,motherage,parentincome
originalid,,,,
10001,Female,1981,NaN,NaN
10002,Male,1982,NaN,NaN
3,Female,1983,26.0,63000.0
4,Female,1981,33.0,11700.0
5,Male,1982,34.0,-3.0
...,...,...,...,...
9018,Female,1980,21.0,-3.0
9019,Male,1984,19.0,-3.0
9020,Male,1980,28.0,83750.0


### 5. merge로 왼쪽 조인
- 첫 번째 데이터(nls97) = 왼쪽 데이터프레임
- 두 번째 데이터(nls97add) = 오른쪽 데이터프레임
- on 매개변수를 사용해 병합 기준 열 지정, how 매개변수 값을 'left'로 설정함

In [133]:
nlsnew = pd.merge(nls97, nls97add, on=['originalid'], how='left')
nlsnew.loc[:, ['originalid','gender','birthyear','motherage','parentincome']]
# nlsnew.loc[nlsnew.originalid>9999, ['originalid','gender','birthyear','motherage','parentincome']]

,originalid,gender,birthyear,motherage,parentincome
0,10001,Female,1981,NaN,NaN
1,10002,Male,1982,NaN,NaN
2,3,Female,1983,26.0,63000.0
3,4,Female,1981,33.0,11700.0
4,5,Male,1982,34.0,-3.0
...,...,...,...,...,...
8979,9018,Female,1980,21.0,-3.0
8980,9019,Male,1984,19.0,-3.0
8981,9020,Male,1980,28.0,83750.0
8982,9021,Male,1980,21.0,23000.0


## 8.2.2 right/inner/right join 

### 1. merge로 오른쪽 조인

In [134]:
nlsnew = pd.merge(nls97, nls97add, on=['originalid'], how='right')
# nlsnew.loc[:, ['originalid','gender','birthyear','motherage','parentincome']]
nlsnew.loc[nlsnew.originalid>9999, ['originalid','gender','birthyear','motherage','parentincome']]

,originalid,gender,birthyear,motherage,parentincome
0,20001,NaN,NaN,26,-3
1,20002,NaN,NaN,19,-4


### 2. merge로 내부 조인

In [135]:
nlsnew = pd.merge(nls97, nls97add, on=['originalid'], how='inner')
# nlsnew.loc[:, ['originalid','gender','birthyear','motherage','parentincome']]
nlsnew.loc[nlsnew.originalid>9999, ['originalid','gender','birthyear','motherage','parentincome']]

,originalid,gender,birthyear,motherage,parentincome


### 3. merge로 외부 조인

In [136]:
nlsnew = pd.merge(nls97, nls97add, on=['originalid'], how='outer')
# nlsnew.loc[:, ['originalid','gender','birthyear','motherage','parentincome']]
nlsnew.loc[nlsnew.originalid>9999, ['originalid','gender','birthyear','motherage','parentincome']]

,originalid,gender,birthyear,motherage,parentincome
0,10001,Female,1981.0,NaN,NaN
1,10002,Male,1982.0,NaN,NaN
8984,20001,NaN,NaN,26.0,-3.0
8985,20002,NaN,NaN,19.0,-4.0


## 8.2.3 ID 불일치 검사
- 왼쪽 데이터프레임, 오른쪽 데이터프레임, 병합 기준 열을 매개변수로 받는 함수 만들기
    - 외부 조인을 수행해서, 병합 기준 값이 한쪽/양쪽 데이터프레임에 모두 있는지 볼 것

In [139]:
def checkmerge(dfleft, dfright, idvar):
    # 왼쪽 데이터프레임 표식열
    dfleft['inleft'] = 'Y'
    # 오른쪽 데이터프레임 표식열
    dfright['inright'] = 'Y'
    # 각 데이터프레임의 idvar과 표식열을, idvar 값 기준으로 외부 조인
    dfboth = pd.merge(dfleft[[idvar, 'inleft']], dfright[[idvar,'inright']], on=[idvar], how='outer')
    dfboth.fillna('N', inplace=True)
    print(pd.crosstab(dfboth.inleft, dfboth.inright))

In [140]:
checkmerge(nls97, nls97add, 'originalid')

inright  N     Y
inleft          
N        0     2
Y        2  8982
